In [48]:
import pandas as pd

### Importa dados do [RefSeq](https://www.ncbi.nlm.nih.gov/genome/browse#!/prokaryotes/)

In [49]:
def carregar_metadados(path:str):
    """Carrega a tabela com os metadados de todas as bactérias presentes no NCBI-RefSeq
    Parametros:
    path -> diretórion onde a tabela de metadados está
    -----
    Returno:
    Um objeto do tipo pandas.DataFrame com as colunas o nome das colunas editados
    """
    table = pd.read_excel(path)
    table.drop(["FTP","RefSeq","FTP.1","rRNA"], axis = 1, inplace = True)
    table.rename(inplace = True, columns = {
    "#Organism":"Especie",
    "Name":"Taxon",
    "Organism":"FTP_genbank",
    "Groups":"FTP_refseq",
    "GenBank":"rRNA"
    })
    table["Genero"] = table.Especie.apply(lambda x: x.split(" ")[0]) 
    return table

In [50]:
refseq_metadados = carregar_metadados("prokaryotes.xlsx")

In [51]:
len(refseq_metadados)

409473

In [52]:
refseq_metadados.dropna(inplace = True)

In [53]:
refseq_metadados = refseq_metadados[refseq_metadados.rRNA > 0]

In [54]:
len(refseq_metadados)

252823

In [55]:
refseq_metadados.head()

,Especie,Taxon,FTP_genbank,FTP_refseq,rRNA,Genero
0,Campylobacter jejuni subsp. jejuni NCTC 11168 ...,Bacteria;Proteobacteria;delta/epsilon subdivis...,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000...,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000...,9,Campylobacter
1,Pseudomonas fluorescens,Bacteria;Proteobacteria;Gammaproteobacteria,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/900...,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/900...,19,Pseudomonas
2,Xanthomonas campestris pv. raphani,Bacteria;Proteobacteria;Gammaproteobacteria,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/013...,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/013...,6,Xanthomonas
3,Salmonella enterica subsp. enterica serovar Ty...,Bacteria;Proteobacteria;Gammaproteobacteria,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000...,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000...,22,Salmonella
4,Yersinia pestis A1122,Bacteria;Proteobacteria;Gammaproteobacteria,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000...,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000...,18,Yersinia


In [56]:
import os
sufix = "_rna_from_genomic.fna.gz"
for info in refseq_metadados[:100].itertuples():
    genome_id = info.FTP_refseq.split("/")[-1]
    os.system(f"wget {info.FTP_refseq + '/' + genome_id + sufix} -O {genome_id}-{info.Genero}.fasta.gz")

In [57]:
os.system(f"mv *.gz data")

0

In [58]:
os.system("gunzip data/*.gz")

0

In [59]:
import glob

In [60]:
rnas = glob.glob("data/*.fasta")

In [61]:
rnas

['data/GCF_000161615.1_ASM16161v1-Bacillus.fasta',
 'data/GCF_016694755.2_ASM1669475v2-Pseudomonas.fasta',
 'data/GCF_000007245.1_ASM724v1-Xylella.fasta',
 'data/GCF_000026745.1_ASM2674v1-Streptococcus.fasta',
 'data/GCF_903886475.1_Streptococcus_thermophilus_CIRM_65-Streptococcus.fasta',
 'data/GCF_020463755.1_ASM2046375v1-Lactococcus.fasta',
 'data/GCF_002076835.1_ASM207683v1-Streptococcus.fasta',
 'data/GCF_900215245.1_IMG-taxon_2617270901_annotated_assembly-Pseudomonas.fasta',
 'data/GCF_013388375.1_ASM1338837v1-Xanthomonas.fasta',
 'data/GCF_013374815.1_ASM1337481v1-Shigella.fasta',
 'data/GCF_002162215.1_ASM216221v1-Histophilus.fasta',
 'data/GCF_000756125.1_ASM75612v1-Burkholderia.fasta',
 'data/GCF_022984195.1_ASM2298419v1-Synechococcus.fasta',
 'data/GCF_006094375.1_ASM609437v1-Staphylococcus.fasta',
 'data/GCF_001932995.2_ASM193299v2-Shigella.fasta',
 'data/GCF_003955815.1_ASM395581v1-Francisella.fasta',
 'data/GCF_000005845.2_ASM584v2-Escherichia.fasta',
 'data/GCF_013177355

In [62]:
len(rnas)

100

In [63]:
from Bio import SeqIO
for arquivo in rnas:
    for info in SeqIO.parse(arquivo,"fasta"):
         if not ("16S" in info.description and "23S" in info.description):
            try:
                rnas.remove(arquivo)
            except:
                pass


In [64]:
len(rnas)

50

In [65]:
from Bio import SeqIO
with open("dataset16s.fasta","w+") as file:
    for arquivo in rnas:
        for info in SeqIO.parse(arquivo,"fasta"):
            if "16S" in info.description:
                genome_id = arquivo.split("/")[-1].split("_r")[0]
                file.write(f">{genome_id}-{info.id}\n")
                file.write(f"{info.seq}\n")           

In [66]:
#Crete db from hmd fasta
!makeblastdb -in /home/tiago/documents/projeto_iam/dataset16s.fasta -out /home/tiago/documents/projeto_iam/bast/rna_db -dbtype nucl -title "rna_16S"



Building a new DB, current time: 06/08/2022 16:41:20
New DB name:   /home/tiago/documents/projeto_iam/bast/rna_db
New DB title:  rna_16S
Sequence type: Nucleotide
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 227 sequences in 0.00981092 seconds.


In [67]:
#Bast my results vs hmd-db
!blastn -db /home/tiago/documents/projeto_iam/bast/rna_db -query /home/tiago/documents/projeto_iam/dataset16s.fasta -outfmt 7 -out rna16s.tsv

In [94]:
rna16s_blast= pd.read_csv(
    "rna16s.tsv", 
    sep = "\t", 
    skiprows = 6,
    names = ["query_id", "subject_id", "identity", "alignment_length", "mismatches", "gap_opens", "q_start", "q_end", "s_start", "s_end", "evalue", "bit_score"]
    )
rna16s_blast.dropna(inplace = True)

In [69]:
rna16s_blast

,query_id,subject_id,identity,alignment_length,mismatches,gap_opens,q_start,q_end,s_start,s_end,evalue,bit_score
0,GCF_016694755.2_ASM1669475v2-Pseudomonas.fasta...,GCF_016694755.2_ASM1669475v2-Pseudomonas.fasta...,100.000,1539.0,0.0,0.0,1.0,1539.0,1.0,1539.0,0.0,2843.0
1,GCF_016694755.2_ASM1669475v2-Pseudomonas.fasta...,GCF_016694755.2_ASM1669475v2-Pseudomonas.fasta...,100.000,1539.0,0.0,0.0,1.0,1539.0,1.0,1539.0,0.0,2843.0
2,GCF_016694755.2_ASM1669475v2-Pseudomonas.fasta...,GCF_016694755.2_ASM1669475v2-Pseudomonas.fasta...,99.805,1539.0,3.0,0.0,1.0,1539.0,1.0,1539.0,0.0,2826.0
3,GCF_016694755.2_ASM1669475v2-Pseudomonas.fasta...,GCF_016694755.2_ASM1669475v2-Pseudomonas.fasta...,99.805,1539.0,3.0,0.0,1.0,1539.0,1.0,1539.0,0.0,2826.0
4,GCF_016694755.2_ASM1669475v2-Pseudomonas.fasta...,GCF_900215245.1_IMG-taxon_2617270901_annotated...,97.987,1540.0,27.0,4.0,1.0,1539.0,1.0,1537.0,0.0,2669.0
...,...,...,...,...,...,...,...,...,...,...,...,...
55565,GCF_002220285.1_ASM222028v1-Bacillus.fasta-lcl...,GCF_000009085.1_ASM908v1-Campylobacter.fasta-l...,76.497,1570.0,289.0,59.0,1.0,1550.0,4.0,1513.0,0.0,782.0
55566,GCF_002220285.1_ASM222028v1-Bacillus.fasta-lcl...,GCF_000009085.1_ASM908v1-Campylobacter.fasta-l...,76.497,1570.0,289.0,59.0,1.0,1550.0,4.0,1513.0,0.0,782.0
55567,GCF_002220285.1_ASM222028v1-Bacillus.fasta-lcl...,GCF_000009085.1_ASM908v1-Campylobacter.fasta-l...,76.497,1570.0,289.0,59.0,1.0,1550.0,4.0,1513.0,0.0,782.0
55568,GCF_002220285.1_ASM222028v1-Bacillus.fasta-lcl...,GCF_000027325.1_ASM2732v1-Mycoplasma.fasta-lcl...,76.489,1578.0,273.0,68.0,7.0,1552.0,7.0,1518.0,0.0,769.0


In [70]:
from Bio import SeqIO
with open("dataset23s.fasta","w+") as file:
    for arquivo in rnas:
        for info in SeqIO.parse(arquivo,"fasta"):
            if "23S" in info.description:
                genome_id = arquivo.split("/")[-1].split("_r")[0]
                file.write(f">{genome_id}-{info.id}\n")
                file.write(f"{info.seq}\n")         

In [71]:
!makeblastdb -in /home/tiago/documents/projeto_iam/dataset23s.fasta -out /home/tiago/documents/projeto_iam/bast/rna23_db -dbtype nucl -title "rna_23S"



Building a new DB, current time: 06/08/2022 16:41:32
New DB name:   /home/tiago/documents/projeto_iam/bast/rna23_db
New DB title:  rna_23S
Sequence type: Nucleotide
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 226 sequences in 0.0135031 seconds.


In [72]:
!blastn -db /home/tiago/documents/projeto_iam/bast/rna23_db -query /home/tiago/documents/projeto_iam/dataset23s.fasta -outfmt 7 -out rna23s.tsv

In [95]:
rna23s_blast= pd.read_csv(
    "rna16s.tsv", 
    sep = "\t", 
    skiprows = 6,
    names = ["query_id", "subject_id", "identity", "alignment_length", "mismatches", "gap_opens", "q_start", "q_end", "s_start", "s_end", "evalue", "bit_score"]
    )
rna23s_blast.dropna(inplace = True)

In [77]:
rna23s_blast.head()

,query_id,subject_id,identity,alignment_length,mismatches,gap_opens,q_start,q_end,s_start,s_end,evalue,bit_score
0,GCF_016694755.2_ASM1669475v2-Pseudomonas.fasta...,GCF_016694755.2_ASM1669475v2-Pseudomonas.fasta...,100.000,1539.0,0.0,0.0,1.0,1539.0,1.0,1539.0,0.0,2843.0
1,GCF_016694755.2_ASM1669475v2-Pseudomonas.fasta...,GCF_016694755.2_ASM1669475v2-Pseudomonas.fasta...,100.000,1539.0,0.0,0.0,1.0,1539.0,1.0,1539.0,0.0,2843.0
2,GCF_016694755.2_ASM1669475v2-Pseudomonas.fasta...,GCF_016694755.2_ASM1669475v2-Pseudomonas.fasta...,99.805,1539.0,3.0,0.0,1.0,1539.0,1.0,1539.0,0.0,2826.0
3,GCF_016694755.2_ASM1669475v2-Pseudomonas.fasta...,GCF_016694755.2_ASM1669475v2-Pseudomonas.fasta...,99.805,1539.0,3.0,0.0,1.0,1539.0,1.0,1539.0,0.0,2826.0
4,GCF_016694755.2_ASM1669475v2-Pseudomonas.fasta...,GCF_900215245.1_IMG-taxon_2617270901_annotated...,97.987,1540.0,27.0,4.0,1.0,1539.0,1.0,1537.0,0.0,2669.0


In [78]:
rna23s_blast.query_id[2]

'GCF_016694755.2_ASM1669475v2-Pseudomonas.fasta-lcl|NZ_CP068034.2_rrna_5'

In [79]:
rna23s_blast.subject_id[2]

'GCF_016694755.2_ASM1669475v2-Pseudomonas.fasta-lcl|NZ_CP068034.2_rrna_84'

In [96]:
rna16s_table = rna16s_blast[["query_id","subject_id","identity"]]

In [97]:
rna16s_table["gene_query"] = rna16s_table.query_id.apply(lambda x: x.split("-")[-1])
rna16s_table["gene_subeject"] = rna16s_table.subject_id.apply(lambda x: x.split("-")[-1])


<ipython-input-97-7f1697763a85>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rna16s_table["gene_query"] = rna16s_table.query_id.apply(lambda x: x.split("-")[-1])
<ipython-input-97-7f1697763a85>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rna16s_table["gene_subeject"] = rna16s_table.subject_id.apply(lambda x: x.split("-")[-1])


In [98]:
rna16s_table["classe"] = rna16s_table.query_id.apply(lambda x: x.split("-")[1].split(".")[0])

<ipython-input-98-bb1a856cac82>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rna16s_table["classe"] = rna16s_table.query_id.apply(lambda x: x.split("-")[1].split(".")[0])


In [100]:
rna16s_table["genome_query"] = rna16s_table.query_id.apply(lambda x: x.split("-")[0])
rna16s_table["genome_subject"] = rna16s_table.subject_id.apply(lambda x: x.split("-")[0])

<ipython-input-100-8bb333c19696>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rna16s_table["genome_query"] = rna16s_table.query_id.apply(lambda x: x.split("-")[0])
<ipython-input-100-8bb333c19696>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rna16s_table["genome_subject"] = rna16s_table.subject_id.apply(lambda x: x.split("-")[0])


In [101]:
rna16s_table

,query_id,subject_id,identity,gene_query,gene_subeject,classe,genome_query,genome_subject
0,GCF_016694755.2_ASM1669475v2-Pseudomonas.fasta...,GCF_016694755.2_ASM1669475v2-Pseudomonas.fasta...,100.000,lcl|NZ_CP068034.2_rrna_5,lcl|NZ_CP068034.2_rrna_14,Pseudomonas,GCF_016694755.2_ASM1669475v2,GCF_016694755.2_ASM1669475v2
1,GCF_016694755.2_ASM1669475v2-Pseudomonas.fasta...,GCF_016694755.2_ASM1669475v2-Pseudomonas.fasta...,100.000,lcl|NZ_CP068034.2_rrna_5,lcl|NZ_CP068034.2_rrna_5,Pseudomonas,GCF_016694755.2_ASM1669475v2,GCF_016694755.2_ASM1669475v2
2,GCF_016694755.2_ASM1669475v2-Pseudomonas.fasta...,GCF_016694755.2_ASM1669475v2-Pseudomonas.fasta...,99.805,lcl|NZ_CP068034.2_rrna_5,lcl|NZ_CP068034.2_rrna_84,Pseudomonas,GCF_016694755.2_ASM1669475v2,GCF_016694755.2_ASM1669475v2
3,GCF_016694755.2_ASM1669475v2-Pseudomonas.fasta...,GCF_016694755.2_ASM1669475v2-Pseudomonas.fasta...,99.805,lcl|NZ_CP068034.2_rrna_5,lcl|NZ_CP068034.2_rrna_53,Pseudomonas,GCF_016694755.2_ASM1669475v2,GCF_016694755.2_ASM1669475v2
4,GCF_016694755.2_ASM1669475v2-Pseudomonas.fasta...,GCF_900215245.1_IMG-taxon_2617270901_annotated...,97.987,lcl|NZ_CP068034.2_rrna_5,lcl|NZ_LT907842.1_rrna_53,Pseudomonas,GCF_016694755.2_ASM1669475v2,GCF_900215245.1_IMG
...,...,...,...,...,...,...,...,...
55565,GCF_002220285.1_ASM222028v1-Bacillus.fasta-lcl...,GCF_000009085.1_ASM908v1-Campylobacter.fasta-l...,76.497,lcl|NZ_CP017060.1_rrna_148,lcl|NC_002163.1_rrna_23,Bacillus,GCF_002220285.1_ASM222028v1,GCF_000009085.1_ASM908v1
55566,GCF_002220285.1_ASM222028v1-Bacillus.fasta-lcl...,GCF_000009085.1_ASM908v1-Campylobacter.fasta-l...,76.497,lcl|NZ_CP017060.1_rrna_148,lcl|NC_002163.1_rrna_8,Bacillus,GCF_002220285.1_ASM222028v1,GCF_000009085.1_ASM908v1
55567,GCF_002220285.1_ASM222028v1-Bacillus.fasta-lcl...,GCF_000009085.1_ASM908v1-Campylobacter.fasta-l...,76.497,lcl|NZ_CP017060.1_rrna_148,lcl|NC_002163.1_rrna_1,Bacillus,GCF_002220285.1_ASM222028v1,GCF_000009085.1_ASM908v1
55568,GCF_002220285.1_ASM222028v1-Bacillus.fasta-lcl...,GCF_000027325.1_ASM2732v1-Mycoplasma.fasta-lcl...,76.489,lcl|NZ_CP017060.1_rrna_148,lcl|NC_000908.2_rrna_4,Bacillus,GCF_002220285.1_ASM222028v1,GCF_000027325.1_ASM2732v1
